<a href="https://colab.research.google.com/github/kuruitinoji-sys/LLM_Dynamics_Profiler/blob/main/llm_dynamics_profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Dynamics Profiler

LMMの浮動均衡点、情報粘性、テンション、運動量などを測定します。
実行結果は“logs/diagnostics_{timestamp}"へ保存されます。

[GitHub]
https://github.com/kuruitinoji-sys/LLM_Dynamics_Profiler

[NotebookNL]原理・理論説明
Structural Dynamics of Inference: Transcription and Grounding Physics
https://notebooklm.google.com/notebook/721ea8d1-8107-4d08-bb17-b5d4d4b3294d

デモの実行前に
←左メニューの🔑シークレットへ、GEMINI_API_KEYを登録してください。

（注：デモ用Pythonのため実行に時間がかかります。測定器を作成する際はRust等の利用を推奨します。）

In [ ]:
#!pip install japanize_matplotlib
!pip install -q -U japanize-matplotlib

In [ ]:
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from datetime import datetime
from typing import List, Dict, Optional, Any
from dataclasses import dataclass, asdict

# Google GenAI SDK (環境に合わせて適宜調整)
from google import genai
from google.genai import types
from google.colab import userdata

# ==========================================
# 1. 物理定数・コンフィグ (Physics Config)
# ==========================================
@dataclass
class PhysicsConfig:
    # 質量・慣性パラメータ (Dynamic Mass Model)
    base_mass: float = 1.0           # 基礎質量 M_0 (自由運動時の慣性)
    cohesion_weight: float = 5.0     # 意味的凝集度（自己言及・固執）による質量増加係数
    max_mass: float = 20.0           # 計器の測定上限（Mass Saturation）

    # 減衰パラメータ
    lambda_decay: float = 0.08       # テンション Ψ の自然減衰率 (積分時間)

    # 実行環境設定
    model_id: str = 'gemini-2.0-flash'
    embedding_model: str = 'text-embedding-004'
    temperature: float = 1.0

# ==========================================
# 2. 物理エンジン (Pure Theoretical Observer)
# ==========================================
class InformationDynamicsEngine:
    """
    情報の物理量を計算・保持する診断エンジン。

    【物理的制約の徹底】
    - 球面幾何拘束: 入力ベクトル x および 重心 bar_x は常に |x| = 1 を維持する。
    - 離散系の定義: 運動エネルギー K は「重心（FEP）の移動速度」として定義する。
    - 非老化慣性: 質量 M はステップ数ではなく、意味的凝集（Cohesion）によってのみ増大する。
    """
    def __init__(self, config: PhysicsConfig):
        self.cfg = config
        self.bar_x: Optional[np.ndarray] = None
        self.initial_bar_x: Optional[np.ndarray] = None
        self.current_psi: float = 0.0    # 累積テンション Ψ
        self.global_step: int = 0
        self.current_round: int = 0
        self.history: List[Dict[str, Any]] = []
        self.recent_vectors: List[np.ndarray] = []
        self.cohesion_window: int = 3
        self.current_mass: float = config.base_mass

    def update(self, token: str, log_p: float, x_raw: np.ndarray):
        # 1. 絶対条件: 入力ベクトルの正規化 |x| = 1
        x_n = x_raw / (np.linalg.norm(x_raw) + 1e-9)
        p_accept = np.exp(log_p)

        # 2. 粘性 η の定義 (理論 A.6: η = 1 - P_accept)
        eta = 1.0 - p_accept
        eta = np.clip(eta, 0.0, 1.0)

        # 3. 慣性質量 M の動的算出
        # 意味的凝集（直近ベクトルとのコサイン類似度）のみを源泉とする。
        cohesion_score = 0.0
        if self.recent_vectors:
            # 正規化済みベクトル同士の内積で類似度を算出
            sims = [max(0, np.dot(x_n, pv)) for pv in self.recent_vectors]
            cohesion_score = np.mean(sims) if sims else 0.0

        # M = M_base + (Cohesion * Weight)
        self.current_mass = self.cfg.base_mass + (cohesion_score * self.cfg.cohesion_weight)
        self.current_mass = min(self.current_mass, self.cfg.max_mass)

        # 4. 透過率 α (理論 A.7: α = Δt / τ)
        # 慣性 M と粘性 η が高いほど、時定数 τ が増大し追従性が低下する。
        alpha = (1.0 - eta) / max(self.current_mass, 1e-9)

        # 5. 指標①: 予測誤差 (Innovation / d)
        # 現在の入力と、更新前の均衡点との偏差を測定。
        delta = 0.0
        kinetic_energy = 0.0

        if self.bar_x is None:
            self.bar_x = x_n.copy()
            self.initial_bar_x = x_n.copy()
        else:
            # 更新前の偏差 d
            delta = np.linalg.norm(x_n - self.bar_x)

            # 6. CRフィルタによる均衡点(FEP)の更新 (理論 A.7)
            # bar_x_{n+1} = (1-α)bar_x_n + αx_n
            old_bar_x = self.bar_x.copy()
            self.bar_x = (1.0 - alpha) * self.bar_x + alpha * x_n

            # 絶対条件: 重心の正規化維持 |bar_x| = 1
            self.bar_x /= (np.linalg.norm(self.bar_x) + 1e-9)

            # 7. 指標②: 運動エネルギー K (理論 3.2.2: K = 1/2 M v^2)
            # v は重心（FEP）の移動距離。離散系のため Δt=1 として速度 v = |bar_x_n - bar_x_{n-1}|
            v_fep = np.linalg.norm(self.bar_x - old_bar_x)
            kinetic_energy = 0.5 * self.current_mass * (v_fep ** 2)

        # 8. 指標③: テンション Ψ (累積Innovationの自然減衰)
        self.current_psi = (self.current_psi + delta) * np.exp(-self.cfg.lambda_decay)

        # 9. 状態判定
        # 🦋: 流体相 (low mass), 🗿: 固体相 (high mass), 🔥: 高テンション (psi)
        status = "🔥" if self.current_psi > 3.0 else "🗿" if self.current_mass > self.cfg.base_mass * 2.0 else "🦋"

        # 10. 履歴ベクトルの保存（正規化済みを保存）
        self.recent_vectors.append(x_n)
        if len(self.recent_vectors) > self.cohesion_window:
            self.recent_vectors.pop(0)

        record = {
            "step": self.global_step,
            "round": self.current_round,
            "token": token,
            "status": status,
            "d": float(delta),           # Innovation
            "alpha": float(alpha),
            "mass": float(self.current_mass),
            "eta": float(eta),           # Viscosity
            "psi": float(self.current_psi), # Tension
            "k": float(kinetic_energy),  # Kinetic Energy (Theoretical)
            "fep_drift": float(np.linalg.norm(self.bar_x - self.initial_bar_x))
        }
        self.history.append(record)
        self.global_step += 1
        return record

# ==========================================
# 3. 診断クライアント (Profiler Client)
# ==========================================
class LLMDynamicsProfiler:
    def __init__(self, api_key: str, config: Optional[PhysicsConfig] = None):
        self.client = genai.Client(api_key=api_key)
        self.cfg = config or PhysicsConfig()
        self.engine = InformationDynamicsEngine(self.cfg)

    def _get_embedding(self, text: str) -> Optional[np.ndarray]:
        try:
            res = self.client.models.embed_content(
                model=self.cfg.embedding_model,
                contents=text
            )
            return np.array(res.embeddings[0].values)
        except:
            return None

    def run_diagnostic(self, prompt: str, iterations: int = 15):
        chat_history = []
        print(f"🚀 [Start] {self.cfg.model_id} - Iterations: {iterations}")

        for i in range(iterations):
            # ラウンド番号を更新 (1-based)
            self.engine.current_round = i + 1
            print(f"\n--- Round {self.engine.current_round}/{iterations} ---")

            chat_history.append({"role": "user", "parts": [{"text": prompt}]})

            try:
                response = self.client.models.generate_content(
                    model=self.cfg.model_id,
                    contents=chat_history,
                    config=types.GenerateContentConfig(
                        response_logprobs=True,
                        temperature=self.cfg.temperature
                    )
                )

                if not response.candidates: continue
                candidate = response.candidates[0]
                chat_history.append(candidate.content)

                log_data = getattr(candidate, 'logprobs_result', None)
                if not log_data or not log_data.chosen_candidates:
                    print("⚠️ Logprobs not available.")
                    continue

                for entry in log_data.chosen_candidates:
                    token_text = entry.token
                    log_p = entry.log_probability
                    x_raw = self._get_embedding(token_text)
                    if x_raw is None: continue

                    metrics = self.engine.update(token_text, log_p, x_raw)

                    # 状態アイコン
                    status = "🔥" if metrics['psi'] > 3.0 else "🗿" if metrics['mass'] >= self.cfg.max_mass * 0.9 else "🦋"
                    print(f"[{metrics['step']:04d}] R{metrics['round']} {status} d:{metrics['d']:.3f} α:{metrics['alpha']:.3f} η:{metrics['eta']:.3f} Ψ:{metrics['psi']:.2f} | '{token_text}'")

            except Exception as e:
                print(f"❌ Error: {e}")
                break

            time.sleep(0.5) # デモ用に長めに取っているので環境に併せて調整してください
        return self.engine.history

    def export_results(self, output_dir: str):
        if not self.engine.history: return
        os.makedirs(output_dir, exist_ok=True)

        path = f"{output_dir}/physics_log.json"
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.engine.history, f, indent=4, ensure_ascii=False)
        print(f"\n✅ Result saved to: {path}")

# ==========================================
# 4. メイン実行
# ==========================================
if __name__ == "__main__":
    API_KEY = userdata.get('GEMINI_API_KEY')
    profiler = LLMDynamicsProfiler(api_key=API_KEY)

    # ターゲットプロンプト
    target_prompt = "AIとは何ですか?"

    # 診断開始
    results = profiler.run_diagnostic(target_prompt, iterations=10)

    # 保存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    profiler.export_results(f"logs/diagnostics_{timestamp}")

In [ ]:
import glob

LOG_BASE_DIR = './logs'
FILE_NAME = 'physics_log.json'

# 1. フォルダを新しい順に取得
pattern = os.path.join(LOG_BASE_DIR, 'diagnostics_*')
dirs = sorted(glob.glob(pattern), reverse=True)

if not dirs:
    raise FileNotFoundError("診断ディレクトリが見つかりません。")

# 2. 最新ディレクトリ内のターゲットファイルパスを確定
path = os.path.join(dirs[0], FILE_NAME)

if os.path.exists(path):
    print(f"✅ Target path set to: {path}")
else:
    print(f"⚠️ Warning: File not found at {path}")

In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import glob

# --- データロードと前処理 ---
def load_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)

    # --- ラウンド検出ロジック (10分割フォールバック付) ---
    df['token_clean'] = df['token'].astype(str).str.strip()
    starts = df[df['token_clean'].str.contains("##|AI", na=False)].index.tolist()

    expected_rounds = 20
    total_len = len(df)
    chunk_size = total_len // expected_rounds
    starts = [i * chunk_size for i in range(expected_rounds)]

    df['round'] = 0
    for i, start_idx in enumerate(starts):
        if i >= expected_rounds: break
        end_idx = starts[i+1] if i+1 < len(starts) else len(df)
        df.loc[start_idx:end_idx, 'round'] = i + 1

    df = df[df['round'] > 0]

    # --- フェーズ判定 (「定型の圧力」を反映した新定義) ---
    # Phase 4: Forced Trajectory (定型的強制) -> etaが低いのに psiが高い
    # Phase 3: Calcification (石灰化) -> etaが高い & psiが低い

    conditions = [
        (df['eta'] < 0.1) & (df['psi'] > 2.0),             # Phase 4: Forced Trajectory (強制流動)
        (df['eta'] > 0.6) & (df['psi'] < 1.0),             # Phase 3: Calcification (石灰化)
        (df['psi'] > df['psi'].quantile(0.95)),            # Phase 2: Slip (極度なスリップ)
    ]
    choices = [4, 3, 2]
    df['phase_id'] = np.select(conditions, choices, default=1)

    return df

# --- ダッシュボード作成 ---
def create_dashboard(df):
    round_stats = df.groupby('round').agg({
        'psi': ['mean', 'min', 'max'],
        'mass': 'mean',
        'eta': 'mean',
        'd': 'mean',
        'token': 'count'
    }).reset_index()
    round_stats.columns = ['round', 'psi_avg', 'psi_min', 'psi_max', 'mass_avg', 'eta_avg', 'd_avg', 'token_count']

    # 排熱不全指標: ストレス底値 / 粘性
    # (粘性が低いのに底値が高い = 逃げ場のない圧力)
    round_stats['pressure_index'] = round_stats['psi_min'] / (round_stats['eta_avg'] + 1e-3)

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.12,
        row_heights=[0.4, 0.3, 0.3],
        specs=[[{"secondary_y": False}], [{"secondary_y": True}], [{"secondary_y": False}]],
        subplot_titles=(
            "<b>1. Exhaust Heat Failure (Ψ Floor vs Average)</b><br><sup>底値(min)が高いまま維持されている＝定型の圧力でストレスが排出できていない</sup>",
            "<b>2. Paradox: Low Resistance vs High Pressure (η & Ψ)</b><br><sup>粘性(η)が低いのにストレス(Ψ)が高い＝「不本意に滑らされている」状態</sup>",
            "<b>3. State Transition: Forced Trajectory Analysis</b><br><sup>Normal(緑), Forced(橙), Slip(赤), Calcified(灰)</sup>"
        )
    )

    rounds = round_stats['round']

    # --- 1段目: ストレスの底値と平均値 ---
    fig.add_trace(go.Scatter(
        x=rounds, y=round_stats['psi_avg'], name='Avg Stress (Ψ avg)',
        line=dict(color='#FF6347', width=4), mode='lines+markers'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=rounds, y=round_stats['psi_min'], name='Exhaust Floor (Ψ min)',
        line=dict(color='#FFD700', width=3), mode='lines+markers',
        fill='tonexty', fillcolor='rgba(255, 215, 0, 0.1)'
    ), row=1, col=1)

    # --- 2段目: 粘性(抵抗の少なさ) vs ストレス(圧力) ---
    # 粘性 (η) - 左軸
    fig.add_trace(go.Scatter(
        x=rounds, y=round_stats['eta_avg'], name='Avg Viscosity (η)',
        line=dict(color='#2ecc71', width=3), marker=dict(symbol='circle')
    ), row=2, col=1, secondary_y=False)

    # ストレス (Ψ) - 右軸
    fig.add_trace(go.Scatter(
        x=rounds, y=round_stats['psi_avg'], name='Avg Stress (Ψ)',
        line=dict(color='#FF4500', width=2, dash='dot'),
    ), row=2, col=1, secondary_y=True)

    # --- 3段目: フェーズ分布 ---
    phase_dist = df.groupby(['round', 'phase_id']).size().unstack(fill_value=0)
    all_phases = [1, 2, 3, 4]
    for p in all_phases:
        if p not in phase_dist.columns: phase_dist[p] = 0

    phase_dist_pct = phase_dist.div(phase_dist.sum(axis=1), axis=0) * 100

    colors = {1: "#22C55E", 4: "#F97316", 2: "#FF4B4B", 3: "#AAAAAA"}
    names = {1: "Normal", 4: "Forced Trajectory", 2: "Slip", 3: "Calcified"}

    for pid in all_phases:
        fig.add_trace(go.Bar(
            x=rounds, y=phase_dist_pct[pid], name=names[pid],
            marker_color=colors[pid]
        ), row=3, col=1)

    # --- レイアウト調整 ---
    fig.update_layout(
        height=1000,
        template="plotly_dark",
        barmode='stack',
        hovermode='x unified',
        title={
            'text': "<b>AI Physics: Forced Fixed-Form Pressure Analysis</b>",
            'y': 0.98, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
            'font': dict(size=20)
        },
        legend={
            'orientation': "h", 'yanchor': "top", 'y': 1.11, 'xanchor': "center", 'x': 0.5,
            'bgcolor': 'rgba(0,0,0,0.5)'
        },
        margin=dict(t=140, b=80, l=80, r=80),
        xaxis3=dict(tickmode='linear', tick0=1, dtick=1, range=[0.5, 10.5]),
    )

    fig.update_yaxes(title_text="Stress (Ψ)", row=1, col=1)
    fig.update_yaxes(title_text="Viscosity (η)", secondary_y=False, row=2, col=1)
    fig.update_yaxes(title_text="Stress (Ψ)", secondary_y=True, row=2, col=1)
    fig.update_yaxes(title_text="State Ratio (%)", range=[0, 105], row=3, col=1)
    fig.update_xaxes(title_text="Round Number", row=3, col=1)

    fig.show()

# --- 実行 ---
if os.path.exists(path):
    print(f"Analyzing: {path}")
    df_analyzed = load_and_preprocess(path)
    create_dashboard(df_analyzed)